## tCoIR - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [2]:
%load_ext autoreload
%autoreload 2

from beakerx.object import beakerx
from beakerx import *

from IPython.display import display #, set_matplotlib_formats
import text_analytic_tools.utility as utility

utility.setup_default_pd_display(pd)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import time
import ipywidgets
import text_analytic_tools.text_analysis.co_occurrence as co_occurrence
import text_analytic_tools.common.text_corpus as text_corpus
from text_analytic_tools.domain_logic_config import current_domain as domain_logic

def compute_co_occurrence(filepath, window_size=5, distance_metric=0, method='HAL', normalize='size'):

    corpus = text_corpus.SimplePreparedTextCorpus(filepath, lowercase=True)
    document_index = domain_logic.compile_documents(corpus)

    df = co_occurrence.compute(corpus, document_index, window_size, distance_metric, normalize, method)

    result_filename = '{}_{}_result_co_occurrence_{}.xlsx'\
        .format(method, window_size, time.strftime("%Y%m%d_%H%M"))
    df.to_excel(result_filename)
    print('Result saved to file {}'.format(result_filename))
    print('Now you are ready to do some serious stuff!')


https://github.com/maciejkula/glove-python/issues/96

```bash
% git clone https://github.com/maciejkula/glove-python.git
% cd glove-python/
% cd glove/
% cython glove_cython.pyx
% cythonize glove_cython.pyx
% cython metrics/accuracy_cython.pyx
% cythonize metrics/accuracy_cython.pyx
% cython --cplus corpus_cython.pyx
% cythonize corpus_cython.pyx
% cd ..
% python setup.py cythonize
% make
% pip install -e .
```

  